In [9]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier,DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import warnings
warnings.filterwarnings('ignore')

In [2]:
spark = SparkSession.builder.appName('mytree').getOrCreate()

22/02/17 11:29:58 WARN Utils: Your hostname, ganesh-pi resolves to a loopback address: 127.0.1.1; using 192.168.1.119 instead (on interface eth0)
22/02/17 11:29:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/17 11:30:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')
data.show()

22/02/17 11:31:14 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [4]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [5]:
#Creating the classifiers
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [6]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [7]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [8]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[98,99,100,1...|    [0.0,1.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[122,123,124...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[129,130,131...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(69

In [12]:
#using an evaluator
acc_eval = MulticlassClassificationEvaluator(predictionCol='prediction', metricName='accuracy')
acc_eval.evaluate(dtc_preds)

0.96875

In [13]:
#Accuracy of GBT trees
acc_eval.evaluate(gbt_preds)

22/02/17 11:39:37 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/02/17 11:39:37 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


0.96875

In [14]:
#Accuracy of RFC trees
acc_eval.evaluate(rfc_preds)

1.0

In [15]:
#getting feature importances
rfc_model.featureImportances

SparseVector(692, {100: 0.0006, 126: 0.0005, 132: 0.0006, 152: 0.0004, 155: 0.0048, 179: 0.0004, 184: 0.0009, 187: 0.0011, 212: 0.0002, 235: 0.012, 240: 0.0007, 242: 0.0001, 243: 0.0018, 245: 0.0102, 259: 0.0038, 260: 0.0014, 262: 0.0068, 263: 0.0006, 266: 0.0006, 268: 0.0004, 271: 0.0006, 272: 0.0064, 273: 0.0075, 290: 0.0077, 291: 0.0033, 299: 0.0025, 300: 0.0147, 301: 0.0028, 302: 0.0048, 314: 0.0026, 317: 0.0084, 322: 0.0018, 325: 0.0005, 327: 0.0006, 328: 0.0086, 329: 0.001, 330: 0.0082, 341: 0.0022, 342: 0.0035, 346: 0.0012, 350: 0.0161, 351: 0.0212, 352: 0.0005, 354: 0.0012, 355: 0.0017, 359: 0.002, 373: 0.0071, 377: 0.0076, 378: 0.0225, 379: 0.0526, 381: 0.002, 384: 0.0029, 385: 0.0066, 399: 0.0062, 400: 0.014, 402: 0.0019, 405: 0.0091, 406: 0.0249, 407: 0.026, 408: 0.0019, 411: 0.0026, 412: 0.0005, 413: 0.0067, 414: 0.0045, 415: 0.0035, 426: 0.0072, 428: 0.0061, 429: 0.0118, 430: 0.0012, 432: 0.0017, 433: 0.0455, 434: 0.0011, 435: 0.0514, 436: 0.0013, 438: 0.0005, 440: 0.012, 

In [ ]:
#Thanks for reading